In [54]:
import pandas as pd
import numpy as np
import datetime as dt

# saved the final dataframe to a new csv file
# changed some of the data types
# the original file was huge so I created a sample of the original data file - 57K sample
df = pd.read_csv('C:/Users/rm2019/PythonProjects/T1.csv',index_col=0)
df.launched = pd.to_datetime(df.launched)
df.deadline = pd.to_datetime(df.deadline)
df.rtdate = pd.to_datetime(df.rtdate)

In [55]:
from sklearn.model_selection import train_test_split

# split the data set into train and test
data = df[['backers','date_diff','goal_usd','name_length']]
y = df.pop('fs_binary')

x_train, x_test, y_train, y_test = train_test_split(data, y,test_size = 0.3)

In [56]:
# data set to test that the kNN is working
#d = {'a':[1,2,3],'b':[4,5,6],'c':[7,8,9],'d':[10,11,12],'e':[1,0,0]}
#e = {'a':[1],'b':[6],'c':[9],'d':[11]}
#dfd = pd.DataFrame(data=d)
#dfe = pd.DataFrame(data=e)
#lbl = dfd.pop('e')
#lbl_t = pd.DataFrame(lbl)
#df_test = pd.DataFrame(dfd[['a','b','c','d']].iloc[0]).T
#lbl_t = pd.DataFrame(lbl_t.iloc[0]).T

In [57]:
y_test.head()

ID
1728443891    0
243488233     1
886891442     0
2116621128    1
1385753664    1
Name: fs_binary, dtype: int64

In [58]:
# the distance formula defined
def distance(df1,df2,labels,k):
    dist_diff = np.sqrt(np.sum(np.square(df1-df2),axis=1))
    dist_diff.name = 'total_dist'
    diff = pd.concat([dist_diff, labels], axis=1)
    diff.sort_values(by='total_dist', axis=0, ascending=True, inplace=True)
    top_knn = diff[:k]
    return top_knn['fs_binary'].value_counts().index.values[0]

In [59]:
# apply the weighted sorted list to the test data
def main():    
    k = 7    
    result_df = x_test.apply(lambda row: distance(row,x_train,y_train,k), axis=1)    
    return result_df
#call to main for the testing
#def main():    
#    k = 2    
#    result_df = dfe.apply(lambda row: distance(row,dfd,lbl,k), axis=1)    
#    return result_df

In [60]:
# assign the output to a dataframe
result = main()

In [72]:
# put the two dataframes together
comb = pd.concat([y_test, result], axis=1)

In [74]:
#rename the columns to original and caluculated
comb.columns = ['original','calculated']

In [96]:
# filter the data and take only those rows where original and calculated are the same
same = comb[comb.original == comb.calculated]
# calculate the percentage of those that are the same
same.count() / comb.count()

original      0.90977
calculated    0.90977
dtype: float64